## 1. Imports

In [11]:
import torch
import numpy as np
import pandas as pd

import sys
sys.path.append('../input/tensorflow-great-barrier-reef')

## 2. Configuration

In [12]:
IOU = 0.50
CONF = 0.15
IMG_SIZE = 1280
AUGMENT = False

CKPT_PATH = '../input/starfish/epoch17.pt'
ROOT_DIR  = '/kaggle/input/tensorflow-great-barrier-reef/'

## 3. Helpers

I don't write any code in this section.

In [13]:
def voc2yolo(bboxes, image_height=720, image_width=1280):
    bboxes = bboxes.copy().astype(float) 
    
    bboxes[..., [0, 2]] = bboxes[..., [0, 2]]/ image_width
    bboxes[..., [1, 3]] = bboxes[..., [1, 3]]/ image_height
    
    w = bboxes[..., 2] - bboxes[..., 0]
    h = bboxes[..., 3] - bboxes[..., 1]
    
    bboxes[..., 0] = bboxes[..., 0] + w/2
    bboxes[..., 1] = bboxes[..., 1] + h/2
    bboxes[..., 2] = w
    bboxes[..., 3] = h
    
    return bboxes

def yolo2coco(bboxes, image_height=720, image_width=1280):
    bboxes = bboxes.copy().astype(float) 
    
    bboxes[..., [0, 2]]= bboxes[..., [0, 2]] * image_width
    bboxes[..., [1, 3]]= bboxes[..., [1, 3]] * image_height
    
    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] - bboxes[..., [2, 3]] / 2
    
    return bboxes

def voc2coco(bboxes, image_height=720, image_width=1280):
    bboxes  = voc2yolo(bboxes, image_height, image_width)
    bboxes  = yolo2coco(bboxes, image_height, image_width)
    
    return bboxes

def predict(model, img, size=768, augment=False):
    height, width = img.shape[:2]
    results = model(img, size=size, augment=augment)  
    preds   = results.pandas().xyxy[0]
    bboxes  = preds[['xmin','ymin','xmax','ymax']].values
    
    if len(bboxes):
        bboxes  = voc2coco(bboxes,height,width).astype(int)
        confs   = preds.confidence.values
        return bboxes, confs
    else:
        return [],[]
    
def format_prediction(bboxes, confs):
    annot = ''
    
    if len(bboxes)>0:
        for idx in range(len(bboxes)):
            xmin, ymin, w, h = bboxes[idx]
            conf             = confs[idx]
            annot += f'{conf} {xmin} {ymin} {w} {h}'
            annot +=' '
            
        annot = annot.strip(' ')
        
    return annot

## 3. Model 

In [14]:
!mkdir -p /root/.config/Ultralytics
!cp /kaggle/input/yolov5-font/Arial.ttf /root/.config/Ultralytics/

In [15]:
def load_model(ckpt_path, conf=0.25, iou=0.50):
    model = torch.hub.load('/kaggle/input/yolov5-lib-ds',
                           'custom',
                           path=ckpt_path,
                           source='local',
                           force_reload=True)  
    model.conf = conf  
    model.iou  = iou  
    model.classes = None 
    model.multi_label = False  
    model.max_det = 1000 
    
    return model

## 4. Prediction and Submission

In [16]:
import greatbarrierreef
env = greatbarrierreef.make_env()
iter_test = env.iter_test()    

In [17]:
model = load_model(CKPT_PATH, conf=CONF, iou=IOU)
for idx, (img, pred_df) in enumerate(iter_test):
    bboxes, confs = predict(model, img, size=IMG_SIZE, augment=AUGMENT)
    annot = format_prediction(bboxes, confs)
    pred_df['annotations'] = annot
    
    env.predict(pred_df)

In [18]:
sub_df = pd.read_csv('submission.csv')
sub_df.head()